In [ ]:
# Authenticate and access Google Cloud services
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import the necessary libraries
from google.cloud import storage
from google.cloud import bigquery

# First make connection to BigQuery
# Replace 'your-project-id' with your Google Cloud project ID
# Replace 'your-dataset-name' with your dataset name
project_id = "whyso-project-0621"
dataset_name = "restaurant_dataset"
table_name = "food_df" # if you make changes previously, then here you need to make according change

bigquery_client = bigquery.Client(project=project_id)

In [ ]:
# Query data based on condition from the table
query = f"""
    SELECT *
    FROM `{project_id}.{dataset_name}.{table_name}`
"""

query_job = bigquery_client.query(query)
results = query_job.result()

restaurant_df = query_job.to_dataframe()

restaurant_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1156    False
1157    False
1158    False
1159    False
1160    False
Length: 1161, dtype: bool

In [ ]:
restaurant_df.head()

,business_id,category_AcaiBowls,category_Bagels,category_Bakeries,category_Barbeque,category_Beer,category_Breakfast_Brunch,category_BubbleTea,category_Buffets,category_Burgers,...,category_Pizza,category_Pretzels,category_Salad,category_Sandwiches,category_Soup,category_Tacos,category_Tapas_SmallPlates,category_Teppanyaki,category_Tonkatsu,category_Waffles
0,UFpCraqzFBAhtZqmxmiWsA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,JWFpjvCc_nkNDVtMPx1ZGg,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,9L-MR0arflwFMF9szEBOOg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,aYMpjij5ShtEoZueMrQPRw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,nTZuJSfwCoNOyA1yS3rbOQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
column_sums = restaurant_df.sum()
column_sums

business_id                       UFpCraqzFBAhtZqmxmiWsAJWFpjvCc_nkNDVtMPx1ZGg9L...
category_AcaiBowls                                                                7
category_Bagels                                                                  11
category_Bakeries                                                                71
category_Barbeque                                                                39
category_Beer                                                                    49
category_Breakfast_Brunch                                                       170
category_BubbleTea                                                               14
category_Buffets                                                                  4
category_Burgers                                                                100
category_Cheesesteaks                                                             1
category_ChickenWings                                                       

In [ ]:
# 选择除 'business_id' 外的所有category相关的列
categories_df = restaurant_df.filter(like='category_')

# 计算每列的总和
column_sums = categories_df.sum()

# 定义阈值
threshold = 10

# 过滤出总和低于阈值的列
columns_to_drop = column_sums[column_sums < threshold].index

# 删除这些列
restaurant_df_edited = restaurant_df.drop(columns=columns_to_drop)

# 显示结果或继续处理
restaurant_df_edited.head()

,business_id,category_Bagels,category_Bakeries,category_Barbeque,category_Beer,category_Breakfast_Brunch,category_BubbleTea,category_Burgers,category_ChickenWings,category_Coffee_Tea,...,category_Fish_Chips,category_HotDogs,category_IceCream_FrozenYogurt,category_JuiceBars_Smoothies,category_Pizza,category_Salad,category_Sandwiches,category_Soup,category_Tacos,category_Tapas_SmallPlates
0,UFpCraqzFBAhtZqmxmiWsA,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,JWFpjvCc_nkNDVtMPx1ZGg,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,9L-MR0arflwFMF9szEBOOg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,aYMpjij5ShtEoZueMrQPRw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,nTZuJSfwCoNOyA1yS3rbOQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
destination_table = 'restaurant_dataset.food_df_edited'
project_id = 'whyso-project-0621'

# Use the to_gbq function to upload the DataFrame (after data processing) to BigQuery
restaurant_df_edited.to_gbq(destination_table, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 5343.06it/s]
